In [1]:
import os
from spark_dbconnect import spark_start, spark_stop

In [7]:
from IPython.display import display
from IPython.core.magic import line_magic, line_cell_magic, Magics, magics_class

@magics_class
class DatabricksConnectMagics(Magics):

   @line_cell_magic
   def sql(self, line, cell=None):
       if cell and line:
           raise ValueError("Line must be empty for cell magic", line)
       try:
           from autovizwidget.widget.utils import display_dataframe
       except ImportError:
           print("Please run `pip install autovizwidget` to enable the visualization widget.")
           display_dataframe = lambda x: x
       return display(self.get_spark().sql(cell or line).toPandas())

   def get_spark(self):
       user_ns = get_ipython().user_ns
       if "spark" in user_ns:
           return user_ns["spark"]
       else:
           from pyspark.sql import SparkSession
           user_ns["spark"] = SparkSession.builder.getOrCreate()
           return user_ns["spark"]

ip = get_ipython()
ip.register_magics(DatabricksConnectMagics)

In [3]:
config = {
    "spark.databricks.service.server.enabled": True,
    "spark.databricks.service.address": "https://adb-2740165739887726.6.azuredatabricks.net",
    "spark.databricks.service.token": os.environ["DATABRICKS_TOKEN"],
    "spark.databricks.service.clusterId": "0802-221206-sols809",
    "spark.databricks.service.orgId": "2740165739887726",
    "spark.databricks.service.port": "15001"
}
spark = spark_start(config=config)

In [4]:
df = spark.read.parquet("dbfs:/FileStore/tables/Broadcast.parq")
df.createOrReplaceTempView("v0")

In [8]:
%%sql
select mmsi,count(1) pop from v0 group by mmsi

,mmsi,pop
0,303105000,499
1,311705000,2466
2,244457000,72
3,369527000,147
4,319101000,2158
...,...,...
738,306156000,202
739,636012507,71
740,367493000,171
741,367070050,189


In [9]:
spark_stop()